# 13. 아이유팬이 좋아할 만한 다른 아티스트 찾기(영화추천)

- CSR matrix가 정상적으로 만들어졌다.
- MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
- 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

In [ ]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data Processaing

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 존재한다
- 별점 데이터는 대표적인 explicit 데이터이나, implicit 데이터로 간주하고 테스트한다.
- 별점을 시청횟수로 해석해서 생각한다. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [3]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'원본 데이터: {orginal_data_size}, 정제된 데이터: {filtered_data_size}')
print(f'남은 데이터 비율 {filtered_data_size / orginal_data_size:.2%}')

원본 데이터: 1000209, 정제된 데이터: 836478
남은 데이터 비율 83.63%


- 16.37% 의 데이터를 삭제한 것을 확인

In [5]:
# ratings 컬럼의 이름을 count로
ratings.rename(columns={'ratings':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [8]:
# 영화 제목 데이터
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
# ratings 와 movies 데이터 프레임 합치기
movie = ratings.join(movies.set_index('movie_id'), on='movie_id')
movie.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [10]:
# 영화 개수
print('영화 개수: ', movie['movie_id'].nunique())

영화 개수:  3628


In [11]:
# 사용자 수
print('사용자 수: ', movie['user_id'].nunique())

사용자 수:  6039


In [12]:
# 인기있는 영화 Top 30
print('인기있는 영화 Top 30')
movie_count = movie.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

인기있는 영화 Top 30


title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [13]:
# 각 유저가 몇 편의 영화를 봤는지 통계량 확인
user_count = movie.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [14]:
# 영화 별 시청(count)횟수 중앙값에 대한 통계
user_median = movie.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

In [17]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('star wars', regex=False)]

,movie_id,title,genre
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2559,2628,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi


In [18]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('star ', regex=False)]

,movie_id,title,genre
122,124,"Star Maker, The (Uomo delle stelle, L') (1995)",Drama
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
325,329,Star Trek: Generations (1994),Action|Adventure|Sci-Fi
790,800,Lone Star (1996),Drama|Mystery
1178,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1192,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1335,1356,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi
1350,1371,Star Trek: The Motion Picture (1979),Action|Adventure|Sci-Fi
1351,1372,Star Trek VI: The Undiscovered Country (1991),Action|Adventure|Sci-Fi
1352,1373,Star Trek V: The Final Frontier (1989),Action|Adventure|Sci-Fi


In [19]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('pulp ', regex=False)]

,movie_id,title,genre
293,296,Pulp Fiction (1994),Crime|Drama


In [20]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('shawshank ', regex=False)]

,movie_id,title,genre
315,318,"Shawshank Redemption, The (1994)",Drama


In [21]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('private ', regex=False)]

,movie_id,title,genre
1119,1135,Private Benjamin (1980),Comedy
1448,1476,Private Parts (1997),Comedy|Drama
1569,1611,My Own Private Idaho (1991),Drama
1959,2028,Saving Private Ryan (1998),Action|Drama|War
3622,3691,Private School (1983),Comedy


In [22]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('space ', regex=False)]

,movie_id,title,genre
65,66,Lawnmower Man 2: Beyond Cyberspace (1996),Sci-Fi|Thriller
667,673,Space Jam (1996),Adventure|Animation|Children's|Comedy|Fantasy
912,924,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
1763,1831,Lost in Space (1998),Action|Sci-Fi|Thriller
1855,1924,Plan 9 from Outer Space (1958),Horror|Sci-Fi
2433,2502,Office Space (1999),Comedy|Romance
2592,2661,It Came from Outer Space (1953),Sci-Fi
2640,2709,Muppets From Space (1999),Children's|Comedy
3757,3827,Space Cowboys (2000),Action|Sci-Fi


In [23]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('incredible ', regex=False)]

,movie_id,title,genre
1002,1015,Homeward Bound: The Incredible Journey (1993),Adventure|Children's
1988,2057,"Incredible Journey, The (1963)",Adventure|Children's


In [24]:
# 좋아하는 영화의 id 리스트
my_favoite_id = [1196, 2028, 318, 296	, 924] # 스타워즈5, 라이언 일병 구하기, 쇼생크탈출, 펄프픽션, 스페이스 오디세이

# id 리스트를 title 리스트로 변환.
my_favorite_movie = []
for mid in my_favoite_id:
    my_favorite_movie.append(list(movies[movies['movie_id']==mid]['title'])[0])
    
# '6041'이라는 임의의 user_id가 위 영화를 5점만점이라고 설정.
my_watchlist = pd.DataFrame({'user_id': ['6041']*5, 'movie_id': my_favoite_id, 'title': my_favorite_movie, 'count':[5]*5})

if not movie.isin({'user_id':['6041']})['user_id'].any():  # user_id에 '6041'이라는 데이터가 없다면
    data = movie.append(my_watchlist)                      # 임의로 만든 my_favorite 데이터를 추가. 

data = data[['user_id', 'movie_id', 'title', 'count']]
data.tail()

,user_id,movie_id,title,count
0,6041,1196,Star Wars: Episode V - The Empire Strikes Back...,5
1,6041,2028,Saving Private Ryan (1998),5
2,6041,318,"Shawshank Redemption, The (1994)",5
3,6041,296,Pulp Fiction (1994),5
4,6041,924,2001: A Space Odyssey (1968),5


In [25]:
# 고유한 유저, 영화를 찾기
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

In [36]:
num_user

6040

In [37]:
num_movie

3628

In [26]:
# 각 키를 인덱스화
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [29]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series구한다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):
    data['new_user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체. 
else:
    print('user_id column indexing Fail')
    
# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print()
    data['new_movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail')

data

,user_id,movie_id,title,count,new_user_id,new_movie_id
0,1,1193,One Flew Over the Cuckoo's Nest (1975),5,0,0
1,1,661,James and the Giant Peach (1996),3,0,1
2,1,914,My Fair Lady (1964),3,0,2
3,1,3408,Erin Brockovich (2000),4,0,3
4,1,2355,"Bug's Life, A (1998)",5,0,4
...,...,...,...,...,...,...
0,6041,1196,Star Wars: Episode V - The Empire Strikes Back...,5,6039,117
1,6041,2028,Saving Private Ryan (1998),5,6039,48
2,6041,318,"Shawshank Redemption, The (1994)",5,6039,157
3,6041,296,Pulp Fiction (1994),5,6039,222


In [30]:
# CSR Matrix 생성
num_user = data['new_user_id'].nunique()
num_movie = data['new_movie_id'].nunique()

# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_data = csr_matrix((data['count'], (data.new_user_id, data.new_movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

- CSR Matrix가 6040x3628로 생성된 것을 확인.

# train

In [31]:
# implicit 라이브러리에서 권장하고 있는 세팅
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [32]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [38]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [41]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [43]:
mine, space =  user_to_idx['6041'], movie_to_idx['2001: A Space Odyssey (1968)']
mine_vector, space_vector = als_model.user_factors[mine], als_model.item_factors[space]

In [44]:
mine_vector

array([ 3.42204005e-01, -1.37475461e-01, -2.91089695e-02,  9.17512700e-02,
        3.60288054e-01, -3.22375655e-01,  9.67408419e-02, -4.28037822e-01,
       -2.53089960e-03, -9.77520198e-02,  9.91620570e-02,  2.65943408e-01,
       -2.45662555e-01, -3.47664922e-01,  3.91660370e-02,  1.89245254e-01,
       -3.59828800e-01, -9.57142487e-02,  5.30774355e-01,  3.02010514e-02,
        1.57282706e-02, -3.60181451e-01,  3.18851054e-01,  3.98368761e-02,
        1.87959448e-02,  4.77468371e-01, -2.12120131e-01,  2.70270526e-01,
        1.46354884e-01, -4.17777032e-01,  3.05600554e-01, -3.97354923e-02,
       -7.25084022e-02, -4.16568443e-02, -1.42566919e-01, -1.64046988e-01,
       -1.06784604e-01, -9.98341367e-02,  1.62798062e-01,  4.62252349e-01,
        2.31461853e-01,  5.50701320e-01,  4.27928269e-01, -1.23383954e-01,
       -3.09755802e-01,  2.68509358e-01,  5.43299392e-02, -2.65072882e-01,
        2.30958700e-01,  4.66254950e-01,  2.62117218e-02, -7.28458613e-02,
        5.22277057e-02, -

In [45]:
space_vector

array([-0.01921556,  0.01807317, -0.00866545,  0.0336165 , -0.00475207,
        0.00768362,  0.01527954, -0.0363647 , -0.02152024, -0.00465219,
        0.01910445,  0.06018637,  0.00851694, -0.02579739, -0.00789285,
       -0.01411365,  0.0015708 , -0.03809979,  0.02845873,  0.00752391,
       -0.02258348, -0.04870445,  0.01265216,  0.0194551 ,  0.03587275,
        0.05869265, -0.01106951,  0.02922349, -0.00364703, -0.05434728,
        0.0388954 ,  0.05294091, -0.0142743 ,  0.03352055,  0.01641319,
        0.01716162, -0.05032219, -0.00160964, -0.00186356,  0.01886438,
       -0.01354413,  0.03911725, -0.01569379, -0.01214196,  0.03245589,
        0.0127113 , -0.02030061,  0.01868403,  0.01735561,  0.020132  ,
        0.01886962, -0.00337913,  0.0117549 ,  0.0111974 ,  0.02099679,
        0.01210708,  0.0082094 , -0.00564433,  0.02033634, -0.0052459 ,
       -0.02099947, -0.0069485 ,  0.01493871, -0.02712357,  0.01802238,
        0.00867028, -0.03505559,  0.03960003,  0.05180554,  0.01

In [46]:
# 나와 스페이스 오디세이의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(mine_vector, space_vector)

0.68503314

In [47]:
# 모델이 나의 titanic 대한 선호도를 어떻게 예측하는지 확인(선호 리스트에 없는 영화)
titanic = movie_to_idx['Titanic (1997)']
titanic_vector = als_model.item_factors[titanic]
np.dot(mine_vector, titanic_vector)

0.046596553

In [48]:
terminator2 = movie_to_idx['Terminator 2: Judgment Day (1991)']
terminator_vector = als_model.item_factors[terminator2]
np.dot(mine_vector, terminator_vector)

0.051695205

In [49]:
def get_similar_artist(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [51]:
get_similar_artist('2001: A Space Odyssey (1968)')

['2001: A Space Odyssey (1968)',
 'Close Encounters of the Third Kind (1977)',
 '2010 (1984)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)',
 'Clockwork Orange, A (1971)',
 'Follow the Bitch (1998)',
 '1-900 (1994)',
 'Blade Runner (1982)',
 'Blowup (1966)',
 'Abominable Snowman, The (1957)']

In [54]:
favorite_movie = '2001: A Space Odyssey (1968)'
movie_id = movies[movies['title']=='2001: A Space Odyssey (1968)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
similar_movie

[(924, 1.0),
 (431, 0.4712427),
 (878, 0.38367957),
 (1488, 0.37183177),
 (228, 0.36734894),
 (1687, 0.36170962),
 (919, 0.36045626),
 (1282, 0.34947473),
 (915, 0.34684303),
 (254, 0.34056103),
 (906, 0.33563215),
 (2381, 0.3354645),
 (922, 0.32533145),
 (1111, 0.32401997),
 (2533, 0.32279715)]

In [55]:
movies[movies['movie_id'].isin([s[0] for s in similar_movie])]

,movie_id,title,genre
225,228,Destiny Turns on the Radio (1995),Comedy
251,254,Jefferson in Paris (1995),Drama
427,431,Carlito's Way (1993),Crime|Drama
867,878,Bye-Bye (1995),Drama
894,906,Gaslight (1944),Mystery|Thriller
903,915,Sabrina (1954),Comedy|Romance
907,919,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical
910,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
912,924,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
1095,1111,Microcosmos (Microcosmos: Le peuple de l'herbe...,Documentary


In [62]:
user = 6039
movie_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
movie_recommended

[(44, 0.6059853),
 (64, 0.5090624),
 (51, 0.4235536),
 (269, 0.41590953),
 (121, 0.4127779),
 (120, 0.36748683),
 (23, 0.33691823),
 (248, 0.25601912),
 (91, 0.24547955),
 (124, 0.22782177),
 (233, 0.21709849),
 (99, 0.20677274),
 (7, 0.1858998),
 (472, 0.17711198),
 (47, 0.17293721)]

In [63]:
movies[movies['movie_id'].isin([m[0] for m in movie_recommended])]

,movie_id,title,genre
6,7,Sabrina (1995),Comedy|Romance
22,23,Assassins (1995),Thriller
43,44,Mortal Kombat (1995),Action|Adventure
46,47,Seven (Se7en) (1995),Crime|Thriller
50,51,Guardian Angel (1994),Action|Drama|Thriller
63,64,Two if by Sea (1996),Comedy|Romance
97,99,Heidi Fleiss: Hollywood Madam (1995),Documentary
118,120,Race the Sun (1996),Drama
119,121,"Boys of St. Vincent, The (1993)",Drama
122,124,"Star Maker, The (Uomo delle stelle, L') (1995)",Drama
